# 1. Construcción de Base de Datos

In [1]:
import mysql.connector

dbConnection = mysql.connector.connect(user='root', host='localhost', password='')
cursor = dbConnection.cursor()

In [2]:
#si no existe la database, no la elimina
try:
    cursor.execute('DROP DATABASE Yapo;')
except mysql.connector.errors.DatabaseError:
    pass

In [3]:
cursor.execute('CREATE DATABASE Yapo;')
cursor.execute('USE Yapo;')

#tabla de usuario
cursor.execute('''CREATE TABLE usuario (
                nombre VARCHAR(100) PRIMARY KEY, ciudad VARCHAR(30),
                codigo_region VARCHAR(3), nombre_region VARCHAR(30));''')

#tabla de anuncio
cursor.execute('''CREATE TABLE anuncio (
                url VARCHAR(300) PRIMARY KEY, titulo VARCHAR(200), descripcion MEDIUMTEXT,
                precio VARCHAR(30), categoria VARCHAR(50), fecha DATE, vendedor VARCHAR(100),
                FOREIGN KEY (vendedor) REFERENCES usuario(nombre));''')

# 2. Scrapping para llenar base de datos

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from pandas import DataFrame

Con el webdriver de la librería selenium, abrimos un navegador Chrome en la página de todos los anuncios de Yapo

In [5]:
browser = webdriver.Chrome()
yapoUrl = 'https://www.yapo.cl/chile/todos_los_avisos?ca=12_s&l=0'
browser.get(yapoUrl)

Con la librería BeautifulSoup, realizamos un scrapping para obtener el url de "última página"

In [6]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

pages = soup.find('span', {'class', 'nohistory FloatRight'}).a['href']

In [7]:
pages

'https://www.yapo.cl/chile?ca=12_s&o=27911'

El parámetro "o" corresponde al número de la página, así que usaremos este para recorrer todas las páginas de anuncios

In [8]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
pages = pages[:index+1]
print(pages)

35
27911
https://www.yapo.cl/chile?ca=12_s&o=


"index"   : corresponde al índice del último caracter "="
"lastPage": corresponde al número de la última página de anuncios
"pages"   : es el enlace "base" que vamos a usar para iterar por todas las páginas de anuncios de chile

In [9]:
for i in range(1, lastPage + 1):
    #recorremos y scrappeamos cada página de anuncios
    url = pages + str(i)
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    links = soup.findAll('td', {'class':'thumbs_subject'})

    #se obtienen todos los links de anuncios en cada página  y luego se scrappea cada una de ellas
    
    for link in links:
        
        #todos los datos que vamos a insertar en la base de datos
        url, title, category, price, date, description, username, regionCode, regionName, city = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'
        
        url = link.a['href']
        
        #vamos a la página del anuncio para obtener toda la información que necesitamos
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        print(url)

        #título del anuncio
        if (soup.find('h1', {'id':'da_subject'})):
            title = soup.find('h1', {'id':'da_subject'}).text.strip()
            print('title:', title)
    
        #categoría del anuncio
        if (soup.find('strong', {'itemprop':'title'})):
            category = soup.find('strong', {'itemprop':'title'}).text.strip()
            print ('category:', category)

        #precio del anuncio
        if (soup.find('div', {'class':'price text-right'}, {'itempop':'price'})):
            price = soup.find('div', {'class':'price text-right'}, {'itempop':'price'}).text.strip('$\n\t').replace('.','')
            print ('price:', price)

        #fecha del anuncio
        if (soup.find('small', {'class':'date'})):
            date = soup.find('small', {'class':'date'}).time['datetime'][:10]
            print('date:', date)

        #descripción del anuncio
        if (soup.find('p', {'itemprop':'description'})):
            description = soup.find('p', {'itemprop':'description'}).text.replace('\n',' ').strip()
            print('description:', description)

        #nombre de usuario del vendedor
        if (soup.find('aside', {'class':'sidebar-right'}).find('seller-info')):
            sellerInfo = soup.find('aside', {'class':'sidebar-right'}).find('seller-info')
            username = sellerInfo['username']
            print('username:', username)
            
        #código de la región, nombre de la región y ciudad
        if (sellerInfo['region']):
            region = sellerInfo['region']
            regionCode = region[:region.find(' ')]
            regionName = region[region.find(' ') + 1:region.find(',')]
            city = region[region.find(',') + 2:]

            print('regionCode:', regionCode)
            print('regionName:', regionName)
            print('city:', city)

        print('\n')

        #se crean las tuplas de cada tabla para insertarlas
        adsTuple = (url, title, description, price, category, date, username)
        usersTuple = (username,city,regionCode,regionName)

        #llenamos la tabla usuario
        try:
            sql = 'INSERT INTO usuario (nombre, ciudad, codigo_region, nombre_region) VALUES (%s, %s, %s, %s);'
            cursor.execute(sql, usersTuple)
        except:
            cursor.execute('ROLLBACK;')
            continue

        #llenamos la tabla anuncio
        try:
            sql = 'INSERT INTO anuncio (url, titulo, descripcion, precio, categoria, fecha, vendedor) VALUES (%s, %s, %s, %s, %s, %s, %s);'
            cursor.execute(sql, adsTuple)
        except:
            cursor.execute('ROLLBACK;')
            continue
            
        cursor.execute("COMMIT;")

https://www.yapo.cl/region_metropolitana/bicicletas_ciclismo/bicicleta_nin__simpson_74076605.htm?ca=15_s&first=1&oa=74076605&xsp=0
title: Bicicleta niñ@ Simpson
category: Bicicletas, ciclismo y accesorios
price: 25000
date: 2020-08-31
description: Bicicleta poco uso. 4-8 años
username: Jacqueline
regionCode: Región
regionName: Metropolitana
city: La Reina


https://www.yapo.cl/valparaiso/moda-vestuario/polera__74076604.htm?ca=6_s&oa=74076604&xsp=1
title: Polera
category: Moda y vestuario
price: 2000
date: 2020-08-31
description: FOOTBALL AMERICANO TALLA L, DETALLE EN UNA MANGA (MANCHA)
username: yarithza avalos
regionCode: V
regionName: Valparaíso
city: Valparaíso


https://www.yapo.cl/region_metropolitana/moda-vestuario/buzo_gap_original_m_y_l_74076603.htm?ca=15_s&oa=74076603&xsp=2
title: Buzo gap original M y L
category: Moda y vestuario
price: 60000
date: 2020-08-31
description: Nuevos y originales solo talla m y L retira en La comuna de San Miguel, Envíos a todo el pais +5694077937

AttributeError: 'NoneType' object has no attribute 'find'

5.3.- Insertar una columna “Perfil” en la tabla Vendedor. Los Vendedores que publicaron más de 5 anuncios en los últimos 7 días, tienen un perfil “Pro”, los otros se consideran “Personal”.

In [13]:
cursor.execute('ALTER TABLE usuario ADD COLUMN perfil VARCHAR(8);')
cursor.execute('UPDATE usuario AS u JOIN (SELECT vendedor, COUNT(vendedor) AS anuncios FROM anuncio GROUP BY vendedor) AS a ON u.nombre = a.vendedor SET u.perfil = if(a.anuncios > 5, \'Pro\', \'Personal\');')

UPDATE usuario AS u JOIN (<br>
&emsp;SELECT vendedor, COUNT(vendedor) AS anuncios<br>
&emsp;FROM anuncio GROUP BY vendedor) AS a<br>
ON u.nombre = a.vendedor<br>
SET u.perfil = if(a.anuncios > 5, \'Pro\', \'Personal\');